In [28]:
import sys, os
import torch
import librosa
import numpy as np
import pandas as pd
from torch import Tensor
from scipy.io import wavfile
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torchaudio
import numpy as np

In [91]:
class RawNetDataset(Dataset):
    def __init__(self, transform=None, mode="train",files_dir=None, base_dir="",csv_file_dir="",nb_time=59049):
        self.base_dir = base_dir
        self.mode = mode
        self.transform = transform
        self.csv_file_dir = csv_file_dir
        self.files_dir = files_dir
        csv_file = pd.read_csv(csv_file_dir,sep="\t")
        self.nb_time = nb_time
        self.classes = np.array([i for i, cls_name in enumerate(csv_file["VoxCeleb1 ID"].unique())])

    def __len__(self):
        return self.classes.shape[0] 
    
    def __getitem__(self, idx):
        filename = self.files_dir[idx]
        X, sample_rate = torchaudio.load(self.base_dir+filename)
        label = self.classes[idx]
        self._pre_emphasis(X)
        nb_time = X.shape[1]
        if nb_time > self.nb_time:
            start_idx = np.random.randint(low = 0,
                high = nb_time - self.nb_time)
            X = X[:, start_idx:start_idx+self.nb_time]
            print("taille supérieure")
        elif nb_time < self.nb_time:
            nb_dup = int(self.nb_time / nb_time) + 1
            X = np.tile(X, (1, nb_dup))[:, :self.nb_time]
            print("taille inférieure")
        else:
            X = X
            print("taille égale")
        return X, label

    def _pre_emphasis(self, x):
        '''
        Pre-emphasis for single channel input
        '''
        return np.asarray(x[:,1:] - 0.97 * x[:, :-1], dtype=np.float32) 



In [97]:
base_dir = 'data/wav'
list_IDs = ['/id10009/7hpSiT9_gCE/00001.wav',
        '/id10009/aFttHpeaXaQ/00001.wav',
        '/id10009/AtavJVP4bCk/00001.wav',
        '/id10009/AtavJVP4bCk/00002.wav',
        '/id10009/AtavJVP4bCk/00003.wav',
        '/id10009/AtavJVP4bCk/00004.wav',
        '/id10009/AtavJVP4bCk/00005.wav',
        '/id10009/AtavJVP4bCk/00006.wav',
        '/id10009/AtavJVP4bCk/00007.wav',
        '/id10009/AtavJVP4bCk/00008.wav',
        ]
csv_file_dir = 'data/target/vox1_meta.csv'

In [98]:
dataset = RawNetDataset(files_dir=list_IDs,base_dir=base_dir,csv_file_dir=csv_file_dir)
data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=1,
                                          shuffle=True)

In [101]:
dataset.__getitem__(5)

taille supérieure


(tensor([[-0.0331, -0.0225, -0.0042,  ...,  0.0011, -0.0005, -0.0031]]), 5)